In [1]:
import pandas as pd
import numpy as np

import sklearn 
from sklearn.feature_extraction.text import TfidfVectorizer

In [2]:
race_df = pd.read_csv('./data/preprocessed_race_tweets.csv', index_col = 0)
race_df

,text,label,user_id
0,ykar futuristic sans serif font,4.0,12488
1,other words good news about the vaccine safety...,4.0,719703
4,how about pizza dipped water,4.0,865071
5,hire better programmers your website dumpster ...,4.0,988211
6,walking home from the adella wonders the raven...,4.0,1025311
...,...,...,...
327593,username danisonbottom wattpad,2.0,3178803853
327594,like going summer shopping today,1.0,3196361888
327596,what the best for guide nutritional needs heal...,4.0,3352812676
327597,freakin panthers,4.0,3924536853


In [3]:
## drop NaN
race_df.dropna(inplace=True)

In [4]:
race_df['label'].value_counts()

4.0    224287
1.0     26269
2.0     16292
3.0      9189
Name: label, dtype: int64

### Sampling from each label
1: Black, 2: Latino/Hspanic, 3: Asian, 4: White

In [5]:
freq = pd.DataFrame({'label':[1, 2, 3, 4],
                     'nostoextract':[9000, 9000, 9000, 9000], })

def bootstrap(data, freq):
    freq = freq.set_index('label')

    # This function will be applied on each group of instances of the same
    # class in `data`.
    def sampleClass(classgroup):
        cls = classgroup['label'].iloc[0]
        nDesired = freq.nostoextract[cls]
        nRows = len(classgroup)

        nSamples = min(nRows, nDesired)
        return classgroup.sample(nSamples)

    samples = data.groupby('label').apply(sampleClass)

    # If you want a new index with ascending values
    # samples.index = range(len(samples))

    # If you want an index which is equal to the row in `data` where the sample
    # came from
    samples.index = samples.index.get_level_values(1)

    # If you don't change it then you'll have a multiindex with level 0
    # being the class and level 1 being the row in `data` where
    # the sample came from.

    return samples

sampled_race_df = bootstrap(race_df,freq)

In [6]:
## stem 
from nltk.stem import PorterStemmer
stemmer = PorterStemmer()

In [7]:
stem_list = []
for txt in sampled_race_df['text']:
    wrds = txt.split()
    stem_wrds = []
    
    for i in wrds:
        stem_wrds.append(stemmer.stem(i))
    
    str1 = ' '.join(stem_wrds)  
    stem_list.append(str1)

sampled_race_df['text'] = stem_list

In [8]:
vectorizer = TfidfVectorizer(stop_words='english', max_features = 5000)

X = vectorizer.fit_transform(sampled_race_df['text'])
print(X.shape)
y = sampled_race_df['label']

(36000, 5000)


### logistic regression + TFIDF vectorizor & cross validation

In [9]:
from sklearn import metrics, preprocessing
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline

In [10]:
from sklearn.metrics import classification_report, accuracy_score, make_scorer, confusion_matrix

def classification_report_with_accuracy_score(y_true, y_pred):
    print(classification_report(y_true, y_pred)) # print classification report
    # print(confusion_matrix(y_true, y_pred, labels = [1,2,3,4], normalize='true'))
    return accuracy_score(y_true, y_pred) # return accuracy score

#### classification report for race

In [11]:
clf = make_pipeline(preprocessing.StandardScaler(with_mean=False), LogisticRegression(max_iter=500))
scores = cross_val_score(clf, X, y, cv=5, \
               scoring=make_scorer(classification_report_with_accuracy_score))

              precision    recall  f1-score   support

         1.0       0.38      0.38      0.38      1800
         2.0       0.33      0.31      0.32      1800
         3.0       0.37      0.42      0.39      1800
         4.0       0.34      0.32      0.33      1800

    accuracy                           0.36      7200
   macro avg       0.36      0.36      0.36      7200
weighted avg       0.36      0.36      0.36      7200

              precision    recall  f1-score   support

         1.0       0.38      0.37      0.38      1800
         2.0       0.34      0.33      0.34      1800
         3.0       0.37      0.41      0.39      1800
         4.0       0.33      0.31      0.32      1800

    accuracy                           0.36      7200
   macro avg       0.36      0.36      0.35      7200
weighted avg       0.36      0.36      0.35      7200

              precision    recall  f1-score   support

         1.0       0.39      0.37      0.38      1800
         2.0       0.

#### classification report for age

In [12]:
age_df = pd.read_csv('./data/preprocessed_tweets_with_for_age_pred.csv',  lineterminator='\n')
## drop NaN
age_df.dropna(inplace=True)

In [13]:
## sample from age_df
freq = pd.DataFrame({'label':[0, 1],
                     'nostoextract':[36115, 36115], })
sampled_age_df = bootstrap(age_df,freq)

In [14]:
all_stem_wrds = []
for txt in sampled_age_df['text']:
    wrds = txt.split()
    stem_wrds = []
    for i in wrds:
        stem_wrds.append(stemmer.stem(i))
    
    str1 = ' '.join(stem_wrds)
    all_stem_wrds.append(str1)

sampled_age_df['text'] = all_stem_wrds

In [15]:
sampled_age_df['label'].value_counts()

0    36115
1    36115
Name: label, dtype: int64

In [16]:
X_age = vectorizer.fit_transform(sampled_age_df['text'])
print(X_age.shape)
y_age = sampled_age_df['label']

(72230, 5000)


In [17]:
scores = cross_val_score(clf, X_age, y_age, cv=5, \
               scoring=make_scorer(classification_report_with_accuracy_score))

              precision    recall  f1-score   support

           0       0.56      0.61      0.58      7223
           1       0.57      0.52      0.54      7223

    accuracy                           0.56     14446
   macro avg       0.56      0.56      0.56     14446
weighted avg       0.56      0.56      0.56     14446

              precision    recall  f1-score   support

           0       0.55      0.62      0.58      7223
           1       0.57      0.50      0.53      7223

    accuracy                           0.56     14446
   macro avg       0.56      0.56      0.56     14446
weighted avg       0.56      0.56      0.56     14446

              precision    recall  f1-score   support

           0       0.56      0.61      0.58      7223
           1       0.57      0.51      0.54      7223

    accuracy                           0.56     14446
   macro avg       0.56      0.56      0.56     14446
weighted avg       0.56      0.56      0.56     14446

              preci

### word2vec + Logistic Regression

In [18]:
from gensim.models import Word2Vec
from gensim.models import KeyedVectors

#### race prediction with word2vec

In [26]:
sents = sampled_race_df['text'].to_list()
sents[:3]

['you play volleybal too you are more than tripl threat your the next level woman',
 'ye and great see real life candyland',
 'everybodi wanna leav facebook for twitter nah keep all ass over there']

In [31]:
model = Word2Vec(sentences=sents[0], vector_size=14)  
str1 = ' '.join(stem_wrds) 
wv = model.wv
wv.key_to_index

{' ': 0, 'e': 1, 'o': 2, 't': 3, 'l': 4, 'a': 5, 'r': 6, 'y': 7}

In [20]:

model = Word2Vec(sentences=sampled_race_df['text'].to_list(), vector_size=500)    
# Store just the words + their trained embeddings.
word_vectors = model.wv
word_vectors.save("word2vec.wordvectors")

In [23]:
# Load back with memory-mapping = read-only, shared across processes.
wv = KeyedVectors.load("word2vec.wordvectors", mmap='r')

print(wv.key_to_index)


{' ': 0, 'e': 1, 't': 2, 'a': 3, 'o': 4, 'i': 5, 'r': 6, 'n': 7, 'h': 8, 'l': 9, 's': 10, 'u': 11, 'd': 12, 'c': 13, 'm': 14, 'w': 15, 'p': 16, 'g': 17, 'y': 18, 'f': 19, 'b': 20, 'k': 21, 'v': 22, 'j': 23, 'x': 24, 'z': 25, 'q': 26, 'ㅋ': 27, 'о': 28, 'е': 29, 'т': 30, 'и': 31, 'ㅠ': 32, 'н': 33, 'с': 34, 'а': 35, 'к': 36, 'п': 37, 'д': 38, 'า': 39, 'い': 40, 'л': 41, 'í': 42, 'р': 43, 'の': 44, 'м': 45, '_': 46, 'た': 47, 'て': 48, 'น': 49, 'é': 50, 'し': 51, 'อ': 52, 'っ': 53, 'な': 54, 'á': 55, 'у': 56, 'か': 57, 'ง': 58, 'ь': 59, 'と': 60, 'ม': 61, 'ก': 62, 'ó': 63, 'に': 64, 'ย': 65, 'เ': 66, 'る': 67, 'で': 68, 'が': 69, 'ด': 70, 'в': 71, 'ñ': 72, 'й': 73, 'ч': 74, 'う': 75, 'ー': 76, 'ล': 77, 'ы': 78, 'だ': 79, 'ン': 80, 'ร': 81, 'ん': 82, 'ا': 83, 'け': 84, 'ш': 85, 'я': 86, 'х': 87, 'แ': 88, '이': 89, 'ว': 90, 'ж': 91, 'з': 92, 'も': 93, 'б': 94, 'ل': 95, 'ト': 96, 'り': 97, 'ะ': 98, 'ま': 99, 'ي': 100, 'れ': 101, 'ไ': 102, 'こ': 103, 'す': 104, 'く': 105, 'を': 106, 'リ': 107, 'は': 108, 'よ': 109, 'ป': 110,

In [22]:
def document_vector(doc, wv = wv):
    """Create document vectors by averaging word vectors. Remove out-of-vocabulary words."""
    doc = [word for word in doc if word in wv.key_to_index]
    print(doc)
    return np.mean(wv[doc], axis=0)

sampled_race_df.loc[:, 'doc_vector']  = sampled_race_df.text.apply(document_vector)

['s', 'e', 'e', ' ', 't', 'h', 'i', ' ', 'i', 'n', 's', 't', 'a', 'g', 'r', 'a', 'm', ' ', 'p', 'o', 's', 't']
['t', 'e', 'l', 'l', ' ', 'a', 'b', 'o', 'u', 't', ' ', 'm', 'a', 'n', ' ', 'h', 'e', 'r', 'e', ' ', 's', 't', 'a', 'r', 'v', ' ', 'a', 's', 's', ' ', 'o', 'f', 'f', ' ', 'l', 'o', 'l']
['h', 'a', 'v', 'e', ' ', 's', 'e', 'c', 'r', 'e', 't', ' ', 's', 'h', 'h', 'h', 'h', ' ', 'd', 'o', 'n', ' ', 't', 'e', 'l', 'l', ' ', 'n', 'o', 'b', 'o', 'd', 'i']
['s', 't', 'r', 'e', 's', 's']
['d', 'a', 'n', 'g', ' ', 'l', 'o', 's', 't', ' ', 't', 'h', 'e', ' ', 'b', 'e', 't', ' ', 't', 'h', 'o', 'u', 'g', 'h', 't', ' ', 'j', 'i', 'n', 't', 'a', 'o', ' ', 'w', 'a', ' ', 'g', 'o', 'n', 'n', 'a', ' ', 'g', 'e', 't', ' ', 'e', 'l', 'e', 'c', 't', ' ', 'b', 'u', 't', ' ', 'a', 'p', 'p', 'a', 'r', ' ', 's', 'o', 'm', 'e', ' ', 'g', 'u', 'y', ' ', 'n', 'a', 'm', 'e', ' ', 'b', 'a', 'r', 'a', 'c', 'k', ' ', 'o', 'b', 'a', 'm', 'a', ' ', 'w', 'o', 'n', ' ', 'i', 'n', 's', 't', 'e', 'a', 'd']
['t',

ValueError: need at least one array to concatenate

In [ ]:
X = sampled_race_df['doc_vector'].to_list()
y = sampled_race_df['label'].to_list()

In [ ]:
clf = make_pipeline(preprocessing.StandardScaler(with_mean=False), LogisticRegression(max_iter=2000, solver='newton'))
scores = cross_val_score(clf, X, y, cv=5, \
               scoring=make_scorer(classification_report_with_accuracy_score))

/Users/yuhao/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:354: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  ConvergenceWarning,


              precision    recall  f1-score   support

         1.0       0.27      0.25      0.26      1800
         2.0       0.31      0.20      0.24      1800
         3.0       0.31      0.21      0.25      1800
         4.0       0.28      0.48      0.36      1800

    accuracy                           0.29      7200
   macro avg       0.29      0.29      0.28      7200
weighted avg       0.29      0.29      0.28      7200



#### Age prediction with word2vec

In [21]:
model = Word2Vec(sentences=sampled_age_df['text'], vector_size=500)    
# Store just the words + their trained embeddings.
word_vectors = model.wv
word_vectors.save("word2vec_age.wordvectors")

# Load back with memory-mapping = read-only, shared across processes.
wv = KeyedVectors.load("word2vec_age.wordvectors", mmap='r')
def document_vector(doc, wv = wv):
    """Create document vectors by averaging word vectors. Remove out-of-vocabulary words."""
    doc = [word for word in doc if word in wv.key_to_index]
    return np.mean(wv[doc], axis=0)
sampled_age_df.loc[:, 'doc_vector']  = sampled_age_df.text.apply(document_vector)

In [25]:
X_age = list(sampled_age_df['doc_vector'])
len(X_age)

1145

In [26]:
scores = cross_val_score(clf, X_age, y_age, cv=5, \
               scoring=make_scorer(classification_report_with_accuracy_score))

              precision    recall  f1-score   support

           0       0.59      0.20      0.30        85
           1       0.66      0.92      0.77       144

    accuracy                           0.65       229
   macro avg       0.62      0.56      0.53       229
weighted avg       0.63      0.65      0.59       229

[[ 17  68   0]
 [ 12 132   0]
 [  0   0   0]]
              precision    recall  f1-score   support

           0       0.57      0.15      0.24        85
           1       0.65      0.93      0.77       144

    accuracy                           0.64       229
   macro avg       0.61      0.54      0.50       229
weighted avg       0.62      0.64      0.57       229

[[ 13  72   0]
 [ 10 134   0]
 [  0   0   0]]
              precision    recall  f1-score   support

           0       0.50      0.24      0.32        85
           1       0.66      0.86      0.74       144

    accuracy                           0.63       229
   macro avg       0.58      0.55   